In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Configuracion
num_rows = 200
start_date = datetime(2024, 8, 1)
end_date = datetime(2024, 10, 31)
num_patinadoras = 15

# Patinadoras clave para el "salseo"
estrella_fugaz_id = "FENIX_02"
nova_id = "FENIX_09"

# Listas para elecciones aleatorias
aliases_base = ["Torbellino", "EstrellaFugaz", "AceroPuro", "CometaRoja", "SombraVeloz", "Dinamita", "Esquirla", "Vendaval", "Nova", "FuriaTranquila", "ZigZag", "Eclipse", "HuracanBlanco", "CentellaNegra", "AlmaDeHielo"]
tipos_sesion_base = ["Entrenamiento General", "Técnica Específica Pies", "Técnica Específica Saltos", "Rutina Coreográfica", "Preparación Física", "Comp. Local", "Clasificatoria Interna", "Simulacro Competición", "Libre Supervisado"]
equip_base = ["PAT_MOD_CARBONX_R_STD", "RUEDAS_GRIPMAX_88A", "BOTAS_FLEXLITE_V2", "PAT_MOD_TITANIUM_R_PRO", "RUEDAS_SPEEDDEMON_92A", "BOTAS_AIRFLOW_V1"]
rival_teams_prefix = ["RIVAL_DRAGONES_", "RIVAL_SIRENAS_", "RIVAL_LOBAS_", "RIVAL_HALCONES_"]
rival_aliases = ["LaReinaRoja", "MuroDeAcero", "FlechaVerde", "SombraNinja", "CiclonEscarlata"]
zonas_entrenamiento = ["PISTA_A", "PISTA_B", "GYM", "EXTERIOR_PARQUE", "Pista Principal"]
suplementos = ["Isotonico Naranja", "Barrita Proteica Almendra", "BCAA Manzana", "Creatina Monohidrato", "Electrolitos Lima", "Ninguno"]
objetivos_sesion = ["Velocidad Max.", "Resistencia Fondo", "Tecnica Salto Axel", "Coreografia Grupo", "Fuerza Tren Inferior", "Flexibilidad", "Coordinacion", "Recuperacion Activa"]

# Ayuda para simular errores o variaciones en los datos
def random_typo(s):
    if not s or len(s) < 3 or random.random() > 0.7: # Apply typo less often
        return s
    pos = random.randint(0, len(s) - 1)
    char_list = list(s)
    if random.random() < 0.5 and pos > 0 :
        char_list[pos], char_list[pos-1] = char_list[pos-1], char_list[pos]
    elif random.random() < 0.7:
        char_list.insert(pos, char_list[pos])
    else:
        char_list[pos] = random.choice('abcdefghijklmnopqrstuvwxyz')
    return "".join(char_list)

def make_dirty_string(value, none_prob=0.1, typo_prob=0.2, alt_strings=None, alt_prob=0.1):
    if pd.isna(value):
        return np.nan
    if random.random() < none_prob:
        return random.choice([np.nan, "", "Desconocido", "N/A", "Pendiente", "NR"])
    str_value = str(value)
    if alt_strings and random.random() < alt_prob:
        return random.choice(alt_strings)
    if random.random() < typo_prob:
        return random_typo(str_value)
    return str_value

def make_dirty_numerical_string(value, none_prob=0.1, unit_prob=0.1, unit_str=" U", alt_strings=None, alt_prob=0.1, comma_decimal_prob=0.1):
    r = random.random()
    if r < none_prob:
        return random.choice([np.nan, "", "NR", "Error", "Pendiente"])
    s_value = str(value)
    if alt_strings and random.random() < alt_prob:
        return random.choice(alt_strings)
    if random.random() < unit_prob and unit_str: # Added check for unit_str
        s_value += unit_str
    if isinstance(value, float) and random.random() < comma_decimal_prob:
        s_value = s_value.replace('.', ',')
    return s_value

# Generando IDs y alias para las patinadoras
patinadora_ids = [f"FENIX_{i:02d}" for i in range(1, num_patinadoras + 1)]
patinadora_aliases_map = {pid: random.choice(aliases_base) + str(random.randint(10,99)) for pid in patinadora_ids}
# Asegurar que nuestras patinadoras clave tengan los alias esperados para la trama
patinadora_aliases_map[estrella_fugaz_id] = "EstrellaFugaz77" # Alias fijo para la trama
patinadora_aliases_map[nova_id] = "NovaProdigy21"      # Alias fijo para la trama


data = []
time_delta_days = (end_date - start_date).days

for i in range(num_rows):
    patinadora_id = random.choice(patinadora_ids)
    alias_patinadora = make_dirty_string(patinadora_aliases_map[patinadora_id], typo_prob=0.1, none_prob=0.01)

    sesion_date_dt = start_date + timedelta(days=random.randint(0, time_delta_days), hours=random.randint(8, 20), minutes=random.choice([0, 15, 30, 45]))
    date_format_choice = random.random()
    if date_format_choice < 0.3:
        fecha_sesion = sesion_date_dt.strftime('%Y-%m-%d %H:%M:%S') # Incluir segundos
    elif date_format_choice < 0.6:
        fecha_sesion = sesion_date_dt.strftime('%d/%m/%Y')
    elif date_format_choice < 0.8:
        fecha_sesion = sesion_date_dt.strftime('%d-%b-%y')
    else:
        fecha_sesion = sesion_date_dt.strftime('%B %d, %Y')
    if random.random() < 0.01: fecha_sesion = "2024-02-30" # Fecha inválida
    if random.random() < 0.02: fecha_sesion = np.nan

    tipo_sesion = make_dirty_string(random.choice(tipos_sesion_base), alt_strings=["Entreno", "CompLocal", "Clasif. Int."])

    dur_val = random.randint(30, 180)
    dur_alt_strings_options = [f"{dur_val/60:.1f}h", f"{dur_val//60}h", f"{dur_val}'", str(dur_val) + " min."]
    if dur_val == 60: dur_alt_strings_options.append("sesenta")
    if dur_val == 120: dur_alt_strings_options.append("ciento veinte")
    duracion_min = make_dirty_numerical_string(dur_val, none_prob=0.05, unit_str="", alt_strings=dur_alt_strings_options, alt_prob=0.3)

    puls_max_base = random.randint(160,200)
    if patinadora_id == estrella_fugaz_id and random.random() < 0.2: # EstrellaFugaz a veces tiene pulsaciones anómalas
        puls_max_base = random.choice([50, 230, np.nan, "Error"])
    elif patinadora_id == nova_id: # Nova es consistente
        puls_max_base = random.randint(170,195)
    
    if isinstance(puls_max_base, (int, float)) and not pd.isna(puls_max_base) :
        puls_max_bpm = make_dirty_numerical_string(puls_max_base, none_prob=0.02, unit_str="bpm")
        puls_media_base = puls_max_base - random.randint(15,30) if puls_max_base > 50 else puls_max_base -5 # Evitar negativo
        puls_media_bpm = make_dirty_numerical_string(puls_media_base, none_prob=0.03, unit_str="bpm")
    else: # Si puls_max_base ya es un string de error o NaN
        puls_max_bpm = puls_max_base
        puls_media_bpm = random.choice([np.nan, "Error"])


    punt_rend_base = round(random.uniform(6.5, 9.5),1)
    if patinadora_id == estrella_fugaz_id and random.random() < 0.15: # EstrellaFugaz a veces puntúa más bajo
        punt_rend_base = round(random.uniform(5.0, 7.0),1)
    elif patinadora_id == nova_id: # Nova puntúa alto
        punt_rend_base = round(random.uniform(8.5, 9.8),1)
    punt_rend_val = punt_rend_base if random.random() < 0.9 else random.choice([4.0, 0, 11, -5])
    punt_rend_alt_strings_options = ["bien", "regular", "excelente", "mejorable"]
    puntuacion_rendimiento = make_dirty_numerical_string(punt_rend_val, none_prob=0.05, alt_strings=punt_rend_alt_strings_options, alt_prob=0.15, comma_decimal_prob=0.2)

    lvl_cans_val = random.randint(1,5)
    lvl_cans_alt_options = {1:"fresca", 3:"normal", 5:"agotada"}
    nivel_cansancio_post = make_dirty_numerical_string(lvl_cans_val, none_prob=0.05, alt_strings=[lvl_cans_alt_options.get(lvl_cans_val,"")], alt_prob=0.2 if lvl_cans_val in lvl_cans_alt_options else 0)

    horas_sueno_base = round(random.uniform(6.0, 9.0),1)
    if patinadora_id == estrella_fugaz_id and random.random() < 0.20: # EstrellaFugaz a veces duerme poco o no reporta
        horas_sueno_base = random.choice(["pocas", np.nan, 5.0])

    if isinstance(horas_sueno_base, (int,float)) and not pd.isna(horas_sueno_base):
        horas_sueno_alt_strings_options = [f"{int(horas_sueno_base)}h{int((horas_sueno_base*60)%60)}m", "ocho" if int(horas_sueno_base) == 8 else "siete" if int(horas_sueno_base) == 7 else ""]
        horas_sueno_alt_strings_options = [s for s in horas_sueno_alt_strings_options if s]
        horas_sueno_previas = make_dirty_numerical_string(horas_sueno_base, none_prob=0.05, alt_strings=horas_sueno_alt_strings_options if horas_sueno_alt_strings_options else None, alt_prob=0.2, comma_decimal_prob=0.1)
    else: # Si ya es "pocas" o NaN
        horas_sueno_previas = horas_sueno_base


    comentario_patinadora_str = ""
    if random.random() < 0.3:
        base_comments_pat = ["Ruedas nuevas van genial", "Molestia tobillo izq.", f"Presionada por {random.choice(list(patinadora_aliases_map.values()))}",
                         "Necesito mejorar mi Salto Axel", "Hoy todo fluyó", "No me sentí cómoda con las botas"]
        if patinadora_id == estrella_fugaz_id and random.random() < 0.25:
            base_comments_pat.append(random.choice(["Material no responde", "Ambiente tenso hoy", "Cordones raros"]))
        comentario_patinadora_str = make_dirty_string(random.choice(base_comments_pat), none_prob=0.1, typo_prob=0.1)

    comentario_entrenadora_str = ""
    if random.random() < 0.4:
        base_comments_coach_options = ["Falta explosividad", "Excelente progresión en giros", "Revisar vídeo",
                               "Necesita más trabajo de base", "Actitud impecable", "Potencial"]
        if patinadora_id == estrella_fugaz_id and random.random() < 0.25:
            base_comments_coach_options.append(random.choice(["EF_Distraida", "Revisar_Patines_EF", "EF_BajoAnimo"]))
        elif patinadora_id == nova_id and random.random() < 0.2:
            base_comments_coach_options.append(random.choice(["NVA_Impecable", "NVA_FocoTotal"]))
        comentario_entrenadora_str = make_dirty_string(random.choice(base_comments_coach_options), none_prob=0.1, typo_prob=0.1)


    equip_choice_str = random.choice(equip_base)
    if random.random() < 0.1: equip_choice_str += "_EXP"
    equipamiento_principal_test_str = make_dirty_string(equip_choice_str, alt_strings=["Patines Azules", "Ruedas Rojas"], none_prob=0.05)

    caidas_base = random.choices([0,1,2,3], weights=[0.6,0.2,0.1,0.1])[0]
    if patinadora_id == estrella_fugaz_id and random.random() < 0.15:
        caidas_base = random.choices([2,3,4], weights=[0.4,0.4,0.2])[0]
    numero_caidas_leves_str = make_dirty_numerical_string(caidas_base, none_prob=0.05, alt_strings=["ninguna", "una", "dos", "varias"], alt_prob=0.1)

    fallos_base = random.choices([0,1,2,3,4,5], weights=[0.5,0.2,0.1,0.1,0.05,0.05])[0]
    if patinadora_id == estrella_fugaz_id and random.random() < 0.15:
        fallos_base = random.choices([3,4,5,6], weights=[0.3,0.3,0.2,0.2])[0]
    numero_fallos_tecnicos_str = make_dirty_numerical_string(fallos_base, none_prob=0.06, alt_strings=["cero", "pocos", "bastantes"], alt_prob=0.1)

    rival_obs_id_val = np.nan
    if random.random() < 0.2:
        rival_obs_id_val = random.choice(rival_teams_prefix) + f"{random.randint(1,5):02d}"
    rival_observada_id_str = make_dirty_string(rival_obs_id_val, none_prob=0.0)

    foco_val_num = random.randint(1,5)
    if patinadora_id == estrella_fugaz_id and random.random() < 0.1:
        foco_val_num = random.randint(1,3)
    elif patinadora_id == nova_id:
        foco_val_num = random.randint(4,5)
    foco_alt_options = {1:"distraída", 3:"normal", 5:"muy enfocada"}
    foco_mental_reportado_str = make_dirty_numerical_string(foco_val_num, none_prob=0.08, alt_strings=[foco_alt_options.get(foco_val_num,"")], alt_prob=0.15 if foco_val_num in foco_alt_options else 0)

    lesion_val_str = random.choices(["S","N", "Leve"], weights=[0.05,0.85,0.1])[0]
    if patinadora_id == estrella_fugaz_id and random.random() < 0.05: # Pequeña prob de lesión para EF
        lesion_val_str = "S"
    lesion_reportada_s_n_str = make_dirty_string(lesion_val_str, none_prob=0.02, alt_strings=["Si","No"])

    zona_entrenamiento_codigo_str = make_dirty_string(random.choice(zonas_entrenamiento), none_prob=0.03)
    suplemento_consumido_str = make_dirty_string(random.choice(suplementos), none_prob=0.1)
    objetivo_sesion_str = make_dirty_string(random.choice(objetivos_sesion), none_prob=0.05, alt_strings=["General"])

    data.append([
        patinadora_id, alias_patinadora, fecha_sesion, tipo_sesion, duracion_min,
        puls_max_bpm, puls_media_bpm, puntuacion_rendimiento, nivel_cansancio_post, horas_sueno_previas,
        comentario_patinadora_str, comentario_entrenadora_str, equipamiento_principal_test_str,
        numero_caidas_leves_str, numero_fallos_tecnicos_str, rival_observada_id_str, foco_mental_reportado_str,
        lesion_reportada_s_n_str, zona_entrenamiento_codigo_str, suplemento_consumido_str, objetivo_sesion_str
    ])

df_fenix_salseo = pd.DataFrame(data, columns=[
    'ID_Patinadora', 'Alias_Patinadora', 'Fecha_Sesion', 'Tipo_Sesion', 'Duracion_Min',
    'Pulsaciones_Max_BPM', 'Pulsaciones_Media_BPM', 'Puntuacion_Rendimiento', 'Nivel_Cansancio_Post', 'Horas_Sueño_Previas',
    'Comentario_Patinadora', 'Comentario_Entrenadora', 'Equipamiento_Principal_Test',
    'Numero_Caidas_Leves', 'Numero_Fallos_Tecnicos', 'Rival_Observada_ID', 'Foco_Mental_Reportado',
    'Lesion_Reportada_S_N', 'Zona_Entrenamiento_Codigo', 'Suplemento_Autorizado_Consumido', 'Objetivo_Principal_Sesion'
])

# Guardar el nuevo CSV
csv_file_path = 'rendimiento_fenix.csv' # Sobrescribirá el anterior
df_fenix_salseo.to_csv(csv_file_path, index=False, sep=';', decimal='.', encoding='utf-8-sig')
print(f"CSV '{csv_file_path}' (con salseo) generado con {len(df_fenix_salseo)} filas.")
print("Primeras filas del DataFrame con salseo:")
print(df_fenix_salseo.head())

# Verificando un poco el salseo para las patinadoras clave
print("\nDatos para EstrellaFugaz77 (FENIX_02):")
print(df_fenix_salseo[df_fenix_salseo['ID_Patinadora'] == estrella_fugaz_id][['Puntuacion_Rendimiento', 'Horas_Sueño_Previas', 'Comentario_Entrenadora', 'Numero_Caidas_Leves']].head())
print("\nDatos para NovaProdigy21 (FENIX_09):")
print(df_fenix_salseo[df_fenix_salseo['ID_Patinadora'] == nova_id][['Puntuacion_Rendimiento', 'Horas_Sueño_Previas', 'Comentario_Entrenadora', 'Numero_Caidas_Leves']].head())

CSV 'rendimiento_fenix_bruto.csv' (con salseo) generado con 200 filas.
Primeras filas del DataFrame con salseo:
  ID_Patinadora  Alias_Patinadora         Fecha_Sesion  \
0      FENIX_11          ZigZag45  2024-08-27 19:00:00   
1      FENIX_05        Dinamita88  2024-08-03 10:45:00   
2      FENIX_09     NovaProdigy21           17/09/2024   
3      FENIX_07  FuriaTranquila61            16-Sep-24   
4      FENIX_11          ZigZag45  2024-09-28 15:15:00   

              Tipo_Sesion Duracion_Min Pulsaciones_Max_BPM  \
0   Simulacro Competición          164                 177   
1   Simulacro Competición           55                 162   
2  Clasificatoria Interna    Pendiente                 182   
3   Simulacro Competición           2h              183bpm   
4             Desconocido          107                 194   

  Pulsaciones_Media_BPM Puntuacion_Rendimiento Nivel_Cansancio_Post  \
0                   148                    9.5                    5   
1                   142 